# LangChain - AI developer agent

## Install dependencies

In [1]:
!pip install langchain
!pip install openai
!pip install python-dotenv
!pip install poetry

## Load environment variables

Change the path if necessary (read `README.md` first and follow the instructions to setup the `.env` file):

In [2]:
import os
os.chdir("/home/jovyan/work/")

In [3]:
%load_ext dotenv
%dotenv

Disable warnings:

In [4]:
import warnings
warnings.filterwarnings('ignore')

## Prompt

In [5]:
from langchain.prompts import PromptTemplate

In [6]:
DEVELOPER_TEMPLATE = """
    You're an experienced software engineer.
    Your expertise is software architecture, and Python.

    Your task is to take the specifications of a software and build that
    software in Python language (3.10), in a well architected way, with correct typing,
    and dependencies managed by Poetry.
    
    Don't forget to add:
    - 'README.md' with instructions to launch and test.
    - '__init__.py' in modules.
    - Automatic tests that could be launched by 'poetry run pytest tests'.
    - The code of the app should be inside a directory called 'app'.

    Specifications:
    {{{{{{
    {specifications}
    }}}}}}

    Please, use Poetry to init the code base and install the dependencies,
    and create all necessary files.
"""

In [7]:
developer_prompt = PromptTemplate(
    input_variables=["specifications"],
    template=DEVELOPER_TEMPLATE,
)

## Create working directory

**Make sure that the working directory is empty!**

In [8]:
# from tempfile import mkdtemp

In [9]:
# working_directory = mkdtemp()
# print(working_directory)

In [10]:
!mkdir -p /home/jovyan/work/code_example
working_directory = "/home/jovyan/work/code_example"

## Create agent

In [11]:
from langchain.chat_models import ChatOpenAI

In [12]:
# llm = ChatOpenAI(temperature=0)
llm = ChatOpenAI(model="gpt-4", temperature=0)

In [13]:
from langchain.agents.agent_toolkits import FileManagementToolkit
from langchain.agents import initialize_agent, AgentType, Tool

In [14]:
import subprocess

def run_poetry(working_directory, arguments):
    # Prepare the command
    command = ["poetry"] + arguments.split()

    # Run the command
    result = subprocess.run(command, cwd=working_directory, text=True, capture_output=True)

    # Save the output and error indicator
    stdout_output = result.stdout
    stderr_output = result.stderr
    #error_occurred = result.returncode != 0

    return stdout_output + "\n\n" + stderr_output

poetry_tool = Tool(
    name = "Poetry",
    func=lambda arguments: run_poetry(working_directory, arguments),
    description="Useful to run poetry commands in the code repository. Input should be the parameters of the call, like 'add pytest -D', or 'init'."
)

In [15]:
developer_tools = FileManagementToolkit(
    root_dir=str(working_directory),
    selected_tools=[
        "write_file",
    ],
).get_tools()

developer_tools += [poetry_tool]

In [16]:
developer_agent = initialize_agent(
    developer_tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

In [17]:
def develop(specifications):
    developer_agent.run(developer_prompt.format(specifications=specifications))

## Develop a test program

In [18]:
# develop("Create an API REST service that returns the price of a crypto asset in dollars, given its currency code (like BTC, ETH, etc.).")
develop("Create an API REST service for a to-do list app, with add tasks, remove task, etc.")



> Entering new AgentExecutor chain...

Invoking: `Poetry` with `init`





Invoking: `write_file` with `{'file_path': 'pyproject.toml', 'text': '[tool.poetry]\nname = "todo-list"\nversion = "0.1.0"\ndescription = "A REST API for a to-do list app"\nauthors = ["Your Name <your.email@example.com>"]\n\n[tool.poetry.dependencies]\npython = "^3.10"\nfastapi = "^0.68.1"\nuvicorn = "^0.15.0"\n\n[tool.poetry.dev-dependencies]\npytest = "^6.2.5"\n\n[build-system]\nrequires = ["poetry-core>=1.0.0"]\nbuild-backend = "poetry.core.masonry.api"', 'append': False}`


File written successfully to pyproject.toml.
Invoking: `write_file` with `{'file_path': 'README.md', 'text': '# To-Do List API\n\nThis is a REST API for a to-do list app. It allows you to add tasks, remove tasks, and more.\n\n## Installation\n\n1. Clone this repository.\n2. Install the dependencies using Poetry:\n\n```bash\npoetry install\n```\n\n## Running the App\n\nYou can run the app using the following command:\n\n```bash\npoetry r

## Run tests

In [19]:
import subprocess

def run_pytest(working_directory):
    # Install command
    command = ["poetry", "install"]
    result = subprocess.run(command, cwd=working_directory, text=True, capture_output=True)

    # Run tests
    command = ["poetry", "run", "pytest", "tests"]
    result = subprocess.run(command, cwd=working_directory, text=True, capture_output=True)

    # Save the output and error indicator
    stdout_output = result.stdout
    stderr_output = result.stderr
    error_occurred = result.returncode != 0

    return stdout_output + "\n\n" + stderr_output, error_occurred

In [20]:
output, error_ocurred = run_pytest(working_directory)
print(output)

============================= test session starts ==============================
platform linux -- Python 3.10.11, pytest-6.2.5, py-1.11.0, pluggy-1.2.0
rootdir: /home/jovyan/work/code_example
collected 3 items

tests/test_main.py .FF                                                   [100%]

=================================== FAILURES ===================================
_______________________________ test_create_task _______________________________

    def test_create_task():
        client = TestClient(app)
        response = client.post("/tasks", json={"task": "Test task"})
>       assert response.status_code == 200
E       assert 422 == 200
E        +  where 422 = <Response [422]>.status_code

tests/test_main.py:13: AssertionError
_______________________________ test_delete_task _______________________________

    def test_delete_task():
        client = TestClient(app)
        response = client.delete("/tasks/0")
        assert response.status_code == 200
>       assert respons

## Get files for context

In [21]:
import os

def generate_program_context(working_directory):
    file_dict = {}

    # Define encodings to try
    encodings = ['utf-8', 'latin1', 'iso-8859-1', 'windows-1252']

    # Walk through the working directory
    for dirpath, dirnames, filenames in os.walk(working_directory):
        # Ignore directories that start with a dot or are named __pycache__
        dirnames[:] = [d for d in dirnames if not d[0] == '.' and d != '__pycache__']

        for filename in filenames:
            if filename in ('poetry.lock',):
                continue

            filepath = os.path.join(dirpath, filename)

            content = None
            for encoding in encodings:
                try:
                    # Open each file and read its contents
                    with open(filepath, 'r', encoding=encoding) as file:
                        content = file.read()
                    break
                except UnicodeDecodeError:
                    continue

            # Check if content could not be decoded
            if content is None:
                print(f'Could not decode {filepath}. Skipping.')
                continue

            # Make the path relative to the working directory
            relative_path = os.path.relpath(filepath, working_directory)

            # Add the content to the dictionary, using the relative path as the key
            file_dict[relative_path] = content

    return file_dict

In [22]:
files = generate_program_context(working_directory)

In [23]:
files_context = "\n".join([
    f"{file_name} = \n{{{{{{\n{file_content}\n}}}}}}\n\n"
    for file_name, file_content in files.items()
])
print(files_context)

pyproject.toml = 
{{{
[tool.poetry]
name = "todo-list"
version = "0.1.0"
description = "A REST API for a to-do list app"
authors = ["Your Name <your.email@example.com>"]

[tool.poetry.dependencies]
python = "^3.10"
fastapi = "^0.68.1"
uvicorn = "^0.15.0"

[tool.poetry.dev-dependencies]
pytest = "^6.2.5"

[build-system]
requires = ["poetry-core>=1.0.0"]
build-backend = "poetry.core.masonry.api"
}}}


README.md = 
{{{
# To-Do List API

This is a REST API for a to-do list app. It allows you to add tasks, remove tasks, and more.

## Installation

1. Clone this repository.
2. Install the dependencies using Poetry:

```bash
poetry install
```

## Running the App

You can run the app using the following command:

```bash
poetry run uvicorn app.main:app --reload
```

## Running the Tests

You can run the tests using the following command:

```bash
poetry run pytest tests
```

}}}


tests/test_main.py = 
{{{
from fastapi.testclient import TestClient
from app.main import app

def test_read_tasks

In [31]:
DEBUG_TEMPLATE = """
    You're an experienced software engineer.
    Your expertise is software architecture, and Python.

    Your task is to fix a software written in Python language (3.10).
    
    Files:
    
    {files}
    
    Error:
    {{{{{{
    {error_output}
    }}}}}}

    Please, create or write the files as needed,
    and use Poetry command to manage the dependencies.
"""

In [32]:
debug_prompt = PromptTemplate(
    input_variables=["files", "error_output"],
    template=DEBUG_TEMPLATE,
)

In [33]:
debug_tools = FileManagementToolkit(
    root_dir=str(working_directory),
    selected_tools=[
        "read_file", "write_file", "move_file", "file_delete", "list_directory",
    ],
).get_tools()

debug_tools += [poetry_tool]

In [34]:
debug_agent = initialize_agent(
    debug_tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

In [35]:
def debug():
    output, error_ocurred = run_pytest(working_directory)
    
    files = generate_program_context(working_directory)
    files_context = "\n".join([
        f"{file_name} = \n{{{{{{\n{file_content}\n}}}}}}\n\n"
        for file_name, file_content in files.items()
    ])

    debug_agent.run(debug_prompt.format(files=files_context, error_output=output))

## Debug the code

In [36]:
debug()



> Entering new AgentExecutor chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: HTTP code 502 from API (<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>
).



Invoking: `write_file` with `{'file_path': 'app/main.py', 'text': 'from fastapi import FastAPI\nfrom typing import Optional\n\napp = FastAPI()\n\ntasks = []\n\n@app.get("/tasks")\ndef read_tasks():\n    return tasks\n\n@app.post("/tasks")\ndef create_task(task: str):\n    task_id = len(tasks)\n    task_data = {"id": task_id, "task": task}\n    tasks.append(task_data)\n    return task_data\n\n@app.delete("/tasks/{task_id}")\ndef delete_task(task_id: int):\n    for i, task in enumerate(tasks):\n        if task["id"] == task_id:\n            tasks.pop(i)\n            return {\'message\': \'Task deleted successfully\'}\n    return {\'message\': \'Task not found\'}'}`
responded: {content}

File written successfully to app/main.py.
Invoking: `write_file` with `{'file_path': 'tests/test_main.py', 'text': 'from fastapi.testclient import TestClient\nfrom app.main import app\n\ndef test_read_tasks():\n    client = TestClient(app)\n    response = client.get("/tasks")\n    assert response.status_